In [4]:
!pip install langchain openai pydantic
!pip install -qU langchain-openai
!pip install -qU pydantic
%pip install --upgrade --quiet langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.8 MB/s eta 0:00:00


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

Step 2: Define the Joke Function Using Pydantic

We'll define a Joke model using Pydantic, which will represent the structure of the joke function.

In [7]:
from pydantic import BaseModel, Field, field_validator

# Define the joke tool using Pydantic
class Joke(BaseModel):
    """Joke to tell the user."""
    setup: str = Field(description="Question to set up a joke")
    punchline: str = Field(description="Answer to resolve the joke")

    # Custom validation: Ensure the setup ends with a question mark
    @field_validator("setup")
    def question_ends_with_question_mark(cls, value):
        if not value.endswith("?"):
            raise ValueError("The setup must be a question ending with '?'!")
        return value


In [8]:
import getpass
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


Step 3: Initialize the OpenAI Model with the Joke Function

We will bind the Joke function to the gpt-3.5-turbo model. This allows the model to use the function calling API to output jokes in a structured format.

In [9]:
from langchain_openai import ChatOpenAI

# Initialize OpenAI model and bind the Joke function
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools([Joke])


Step 4: Create the ChatPromptTemplate

We will now define a prompt template, which represents the system and user instructions that are passed to the model.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

# Create a prompt template with system and user instructions
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."), ("user", "{input}")]
)


Step 5: Implement Different Output Parsers

We'll start by using the JsonOutputToolsParser to extract the function's output in a structured JSON format. Later, we will explore the PydanticToolsParser.
5.1 Using JsonOutputToolsParser

In [11]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser

# Initialize the JSON parser
parser = JsonOutputToolsParser()

# Combine prompt, model, and parser into a chain
chain = prompt | model | parser

# Run the chain to invoke the model with a joke request
result = chain.invoke({"input": "tell me a joke"})
print(result)


[{'args': {'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was two tired!'}, 'type': 'Joke'}]


5.2 Adding the return_id=True Option

If you want to include the tool call ID along with the result, you can modify the parser:

In [12]:
# Initialize the JSON parser with return_id=True
parser = JsonOutputToolsParser(return_id=True)

# Combine everything into a chain
chain = prompt | model | parser

# Run the chain again
result = chain.invoke({"input": "tell me a joke"})
print(result)


[{'args': {'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was two tired!'}, 'id': 'call_LIjaPLxygeWwaFFXhXXBPHqn', 'type': 'Joke'}]


5.3 Using JsonOutputKeyToolsParser (Extracting Specific Keys)

If you only want to extract the specific keys from the response (e.g., the setup and punchline), you can use the JsonOutputKeyToolsParser.

In [13]:
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

# Initialize the parser to extract only the "Joke" key
parser = JsonOutputKeyToolsParser(key_name="Joke")

# Combine prompt, model, and parser into a chain
chain = prompt | model | parser

# Run the chain
result = chain.invoke({"input": "tell me a joke"})
print(result)


[{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was two tired!'}]


Step 6: Using PydanticToolsParser for Validation

Lastly, let’s use the PydanticToolsParser to validate the function's output using the Joke model.

In [14]:
from langchain.output_parsers.openai_tools import PydanticToolsParser

# Initialize the Pydantic parser with the Joke model
parser = PydanticToolsParser(tools=[Joke])

# Create the model chain
chain = prompt | model | parser

# Run the chain
result = chain.invoke({"input": "tell me a joke"})
print(result)


[Joke(setup="Why couldn't the bicycle stand up by itself?", punchline='Because it was two tired!')]
